In [1]:
import pandas as pd
import pygwalker as pyg

In [2]:
order_df = pd.read_csv('/Users/raphaelseo/Documents/study/fastcampus/바이트디그리/최종/order-9968.txt')
course_df = pd.read_csv('/Users/raphaelseo/Documents/study/fastcampus/바이트디그리/최종/course-9968.csv')
customer_df = pd.read_excel('/Users/raphaelseo/Documents/study/fastcampus/바이트디그리/최종/customer-9968.xlsx', engine='openpyxl')
refund_df = pd.read_csv('/Users/raphaelseo/Documents/study/fastcampus/바이트디그리/최종/refund-9968.csv')
user_df = pd.read_csv('/Users/raphaelseo/Documents/study/fastcampus/바이트디그리/최종/user-9968.csv')

### Q2-1
- 2022년 1월 중 가장 매출이 높았던 3일과 낮았던 3일은 언제인가요? (status = COMPLETED 기준)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
# 주문 데이터중 상태가 Complete인 것들만 추출 및 날짜데이터 변환
target = order_df[order_df['state']=='COMPLETED']
target['created_at'] = pd.to_datetime(target['created_at']).dt.normalize()

In [14]:
# 매출 기준을 세전 가격인 tax_free_price를 기준으로 정하여 월별 합산을 계산
t = target.groupby('created_at')['tax_free_price'].sum().reset_index()
# 타겟 데이터 중 1월 데이터만 추출
filtered_data = t[t['created_at'].dt.month == 1]
# 데이터를 매출 기준 내림차순으로 정렬
sort_data = filtered_data.sort_values('tax_free_price', ascending=False)

In [15]:
pyg.walk(sort_data)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc0a920c48hEwDGvJ8xSBr0CWI" style="height: auto">\n    <head>…

#### Q2-2
- (Order) 그리고 그 때의 매출액(list_price)과, 주문 수, 신청자 수, 강의 수는 각각 얼마인가요?

In [16]:
high_date = sort_data['created_at'][:3]
low_date = sort_data['created_at'][-3:]
user_count = target.groupby('created_at')['customer_id'].nunique().reset_index()
user_count = user_count.rename(columns={'customer_id':'num of customer'})
user_count[(user_count['created_at'].isin(high_date)) | (user_count['created_at'].isin(low_date))]
order_count = target.groupby('created_at')['id'].nunique().reset_index()
order_count = order_count.rename(columns={'id':'num of order'})
order_count[(order_count['created_at'].isin(high_date)) | (order_count['created_at'].isin(low_date))]

,created_at,num of order
0,2022-01-01,238
1,2022-01-02,412
4,2022-01-05,1276
8,2022-01-09,1195
21,2022-01-22,217
30,2022-01-31,902


In [17]:
course_count = target.groupby('created_at')['name'].nunique().reset_index()
course_count = course_count.rename(columns={'id':'num of course'})
course_count[(course_count['created_at'].isin(high_date)) | (course_count['created_at'].isin(low_date))]

,created_at,name
0,2022-01-01,91
1,2022-01-02,124
4,2022-01-05,172
8,2022-01-09,161
21,2022-01-22,105
30,2022-01-31,165


##### 각 일자별 신청자, 강의, 주문 수
- 매출이 가장 높았던 9, 5, 31일의 신청자, 강의, 주문 수 모두 낮은 날들에 비해 비교적 높은 수치를 보여주고있습니다. 

#### Q2-3 (Order)
- 월별 각 강의 별 할인율은 어떻게 될까요?
- 할인율의 경우 list price 와 discount price를 활용하여 측정하였습니다.

In [18]:
target = order_df.copy()
target['created_at'] = pd.to_datetime(target['created_at']).dt.month
target['discount_rate'] = target['discount_price']/target['list_price']*100

In [19]:
target2 = target.groupby(['name', 'created_at'])['discount_rate'].mean().reset_index().sort_values('discount_rate', ascending=False)

In [20]:
pyg.walk(target2)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc0b661d154cXGvoF9Dby7frjw" style="height: auto">\n    <head>…

#### Q2-4 
- (Course) 강의 키워드에서 가장 많이 언급된 단어는 무엇인가요?

In [21]:
from collections import Counter

In [22]:
target = course_df[course_df['keywords'].notnull()]
res = ','.join(target['keywords'])
ls = res.split(',')
result = Counter(ls)
res_dict = {'keywords':result.keys(), 'count':result.values()}
result_df = pd.DataFrame(res_dict)

In [23]:
pyg.walk(result_df)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc0cc4263bRn5Fy2zAtGK3wEWa" style="height: auto">\n    <head>…

#### Q2-5 
- (Refund) 환불이 가장 많은 강의는 어떤 강의였나요?

In [24]:
target = refund_df.groupby('course_id').size().sort_values(ascending=False).reset_index()

In [25]:
target = target.rename({0:'count'}, axis=1)

In [26]:
pyg.walk(target)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc0ed69a12EabRwl5m0XtCqMeW" style="height: auto">\n    <head>…

#### Q2-6 
- (Customer) 강의를 가장 많이 신청한 고객과 그 금액은 얼마인가요?

In [27]:
target = order_df[order_df['state']=='COMPLETED']
target2 = target.groupby('customer_id').size().sort_values(ascending=False).reset_index()
target2 = target2.rename({0:'count'},axis=1)

In [28]:
pyg.walk(target2)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc11075441kT0tEZgqrNHwiSWb" style="height: auto">\n    <head>…

In [29]:
target2 = target.groupby('customer_id')['tax_free_price'].sum().sort_values(ascending=False).reset_index()
target2 = target2.rename({0:'sum'}, axis=1)

In [30]:
pyg.walk(target2)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc1342727fS3YfHjuLcxlIWtmn" style="height: auto">\n    <head>…

#### Q2-7
- (Order) 주말과 주중의 평균 강의 신청율은 어떻게 다를까요?

In [31]:
target = order_df.copy()
target['created_at'] = pd.to_datetime(target['created_at']).dt.day_name()

In [32]:
weekend = target[(target['created_at']=='Saturday') | (target['created_at']=='Sunday')]

In [33]:
weekday = target[(target['created_at']!='Saturday') & (target['created_at']!='Sunday')]

In [34]:
pyg.walk(target)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc14717042NPQUy8ZwT4H79EFx" style="height: auto">\n    <head>…

#### Q2-8
- (User) 주로 몇시에 회원 가입(created_at)이 활발했나요? 월별로 차이가 있는지 보여주고, 그 근거를 추론해 보세요

In [35]:
target = user_df.copy()
target['hour'] = pd.to_datetime(target['created_at']).dt.hour
target['month'] = pd.to_datetime(target['created_at']).dt.month

In [36]:
target = target.groupby('month')['hour'].agg(lambda x: x.mode()).reset_index()

In [37]:
pyg.walk(target)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc201a5cccCZMIKS7J9UfskEmn" style="height: auto">\n    <head>…

### Q2-9
- 강의 가격(list_price)을 5만원 단위의 도수분포표로 나타내 주세요.

In [61]:
# 가격 데이터를 50000 단위로 분리, 각 단위별로 그룹화하여 사이즈 계산
bins = range(0, order_df['list_price'].max() + 50000, 50000)
traget = order_df.copy()
traget['bin'] = pd.cut(traget['list_price'], bins=bins, right=False)
freq_table = traget.groupby('bin').size().reset_index(name='frequency')

In [62]:
# 갯수가 0이 아닌 데이터만을 추출
nonzero_table = freq_table[freq_table['frequency'] != 0]

In [65]:
nonzero_table

,bin,frequency
0,"[0, 50000)",267266
1,"[50000, 100000)",12565
2,"[100000, 150000)",3656
3,"[150000, 200000)",73
4,"[200000, 250000)",172
5,"[250000, 300000)",4
6,"[300000, 350000)",52
7,"[350000, 400000)",202
8,"[400000, 450000)",5
9,"[450000, 500000)",4


### Q2-10
- 신청이 완료된 강의(complated)와 진행중인 강의(pending), 그리고 취소된 강의(cancelled) 간의 금액(list_price)과 유저의 비중을 월별로 각각 구하시오?

In [47]:
# 주문 데이터를 주문의 상태별로 분류 및 월 데이터 생성
target = order_df.copy()
target['month'] = pd.to_datetime(target['created_at']).dt.month
comp = target[target['state']=='COMPLETED']
pend = target[target['state']=='PENDING']
canc = target[target['state']=='CANCELLED']

In [48]:
pyg.walk(target)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc267f2f4bcLxnQeKTC7UAEw9y" style="height: auto">\n    <head>…

### Q2-11
- state이 HIBERNATED 된 Used들은 ID를 생성(created_at)하고 마지막 로그인(last_login_at)까지 평균적으로 몇일이 걸렸나요?

In [49]:
# 유저 데이터 중 상태가 HIBERNATED인 데이터를 추출
target = user_df[user_df['state']=='HIBERNATED']

In [50]:
# 데이터의 생성 시간과 마지막 로그인 시간의 날짜를 시간을 제외하고 변환
target['created_at'] = pd.to_datetime(target['created_at']).dt.normalize()
target['last_login_at'] = pd.to_datetime(target['last_login_at']).dt.normalize()

In [51]:
# 데이터별 생성 날짜와 마지막 로그인 날짜의 차이를 계산
target['diff'] = target['last_login_at'] - target['created_at']

In [52]:
# 차이의 평균을 계산
target['diff'].mean()

Timedelta('154 days 11:45:04.321257092')

## Trend Analysis

### Order data trend
- order data의 tax free price를 활용하여 월별 매출 트렌드를 시각화 하였습니다.

In [53]:
# 각 데이터의 날짜 데이터에서 월 데이터 추출
order_df['month'] = pd.to_datetime(order_df['created_at']).dt.month
course_df['month'] = pd.to_datetime(course_df['created_at']).dt.month
user_df['month'] = pd.to_datetime(user_df['updated_at']).dt.month
customer_df['month'] = pd.to_datetime(customer_df['created_at']).dt.month
refund_df['month'] = pd.to_datetime(refund_df['created_at']).dt.month

In [54]:
# 주문 데이터의 날짜에서 연도를 추출하여 이를 바탕으로 2022년도 데이터만 분리
order_df['year'] = pd.to_datetime(order_df['created_at']).dt.year
target = order_df[order_df['year']==2022]
order_trend = target.groupby('month')['tax_free_price'].sum().reset_index()

In [55]:
pyg.walk(target)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc2befa17fjR3DWKNqEZahV2Mf" style="height: auto">\n    <head>…

### Course data trend 분석
- 강의 데이터의 created_at 데이터를 활용하여 월별 강의의 생성 수를 분석

In [ ]:
# 강의 데이터의 날짜에서 연도를 추출하여 이를 바탕으로 2022년도 데이터만 분리
course_df['year'] = pd.to_datetime(course_df['created_at']).dt.year
target = course_df[course_df['year']==2022]
target = target.groupby(['month']).size().reset_index()

In [56]:
pyg.walk(target)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc39e0a3db5CQzIX3J2mtdOyl0" style="height: auto">\n    <head>…

### User data trend 분석
- 유저 데이터의 업데이트 시간과 유저의 상태를 확인하여 삭제된 인원수 비교
- 삭제된 유저의 마지막 업데이트 시간을 삭제 시간으로 추정

In [73]:
# 유저 데이터의 날짜에서 연도를 분리하고 삭제한 유저를 추출, 추출된 데이터를 월별로 그룹화하여 사이즈를 계산
user_df['year'] = pd.to_datetime(user_df['updated_at']).dt.year
target = user_df[(user_df['year']==2022) & (user_df['state']=='DELETED')]
target2 = target.groupby('month').size().reset_index()
target2 = target2.rename({0:'count'}, axis=1)

In [74]:
pyg.walk(target2)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc7e20674atlLTUz2Y18muQXxj" style="height: auto">\n    <head>…

### Customer data trend 분석
- 소비자 데이터의 생성 시간을 활용하여 월별 회원가입한 인원수를 비교

In [82]:
# 소비자 데이터의 날짜에서 연도를 분리하고 2022년 데이터로 한정, 월별로 그룹화하여 사이즈 계산
customer_df['year'] = pd.to_datetime(customer_df['created_at']).dt.year
target = customer_df[customer_df['year']==2022]
target2 = target.groupby('month').size().reset_index()
target2 = target2.rename({0:'count'}, axis=1)

In [83]:
pyg.walk(target2)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc827b35ceH2FJxIiMckNmbC5D" style="height: auto">\n    <head>…

### Refund data trend 분석
- 환불 데이터를 활용하여 월별 환불금액을 비교

In [85]:
# 환불 데이터의 날짜에서 연도를 분리하고 2022년 데이터로 한정, 월별로 그룹화하여 환불금액 계산
refund_df['year'] = pd.to_datetime(refund_df['created_at']).dt.year
target = refund_df[refund_df['year']==2022]
target2 = target.groupby('month')['tax_free_amount'].sum().reset_index()
target2['tax_free_amount'] = target2['tax_free_amount'] * -1

In [86]:
pyg.walk(target2)

Box(children=(HTML(value='<div id="ifr-pyg-00061bcc8489f27bmZeRM4Nrian5KJlv" style="height: auto">\n    <head>…